In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import os
import torchaudio
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Model, Wave2Vec2Processor, Trainer, TrainingArguments, Wave2Vec2ForSequenceClassification

import warnings
warnings.filterwarnings("ignore")

In [ ]:
paths = []
labels = []

for dirname, _, filenames in os.walk("/input"):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split("_")[-1]
        label = label.split(".")[0]
        labels.append(label.lower())
    
    if len(paths) == 2800:
        break

In [ ]:
df = pd.DataFrame()
df["audio_paths"] = paths
df["labels"] = labels

### Exploratory Data Analysis

In [ ]:
sns.countplot(data=df, x="labels")

In [ ]:
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10,4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data, sr=sr)
    plt.show()

def spectogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.aplitude_to_db(abs(x))
    plt.figure(figsize=(11,4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis="time", y_axis="hz")
    plt.colorbar()

In [ ]:
emotion = "fear"
path = np.array(df["audio_paths"][df["labels"]==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectogram(data, sampling_rate, emotion)
Audio(path)